In [3]:
## Connecting All CSV files

import os
from dotenv import load_dotenv

load_dotenv()


True

In [5]:
import os
os.getcwd()
os.listdir()


['.ipynb_checkpoints',
 'crypto_market_data_with_dates.csv',
 'market_project.ipynb.ipynb',
 'oil_prices_2020_2026_inr.csv',
 'stock_prices_2020_2025_clean_inr.csv',
 'top_3_historical_prices_inr.csv']

In [6]:
import pandas as pd

crypto = pd.read_csv("crypto_market_data_with_dates.csv")
oil = pd.read_csv("oil_prices_2020_2026_inr.csv")
stock = pd.read_csv("stock_prices_2020_2025_clean_inr.csv")
top3 = pd.read_csv("top_3_historical_prices_inr.csv")

print("All files loaded successfully!")


All files loaded successfully!


In [7]:
# Re-establishing connection for the queries
conn = mysql.connector.connect(host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, database="market_db")
cursor = conn.cursor()

def run_query(title, sql):
    print(f"\n--- {title} ---")
    df = pd.read_sql(sql, conn)
    print(df)

# 1. Top 3 by Market Cap
run_query("Top 3 Cryptos by Market Cap", 
         "SELECT name, market_cap FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 3")

# 2. Bitcoin % Change (Sep 2024 vs Sep 2025)
# Note: This assumes you have data for these specific dates
run_query("BTC % Change (Sep 24 to Sep 25)", """
    SELECT 
        ((max_p.price_usd - min_p.price_usd) / min_p.price_usd) * 100 as percentage_change
    FROM 
        (SELECT price_usd FROM crypto_prices WHERE coin_id='bitcoin' AND date LIKE '2024-09%') as min_p,
        (SELECT price_usd FROM crypto_prices WHERE coin_id='bitcoin' AND date LIKE '2025-09%') as max_p
    LIMIT 1
""")

# 3. Supply > 90% of Total
run_query("High Circulating Supply (>90%)", 
         "SELECT name, symbol FROM cryptocurrencies WHERE circulating_supply > (0.9 * total_supply)")


--- Top 3 Cryptos by Market Cap ---
       name       market_cap
0   Bitcoin  148399525448423
1  Ethereum   27888870353563
2    Tether   16971443086480

--- BTC % Change (Sep 24 to Sep 25) ---
Empty DataFrame
Columns: [percentage_change]
Index: []

--- High Circulating Supply (>90%) ---
                             name symbol
0                             0xy    0xy
1                           1INCH  1inch
2                 A Hunters Dream    caw
3                            A7A5   a7a5
4                            Aave   aave
..                            ...    ...
749                 Zebec Network   zbcn
750                          Flux   flux
751             Zeus Network zBTC   zbtc
752                       Zilliqa    zil
753  zkSync Bridged USDC (zkSync)   usdc

[754 rows x 2 columns]


C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\1767411544.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\1767411544.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\1767411544.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [8]:
import mysql.connector
import pandas as pd

# 1. Connect
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD"),
    database="market_db"
)

def run_task(title, sql):
    print(f"\n{'='*50}\n{title}\n{'='*50}")
    try:
        df = pd.read_sql(sql, conn)
        if df.empty:
            print("No data found for this specific criteria.")
        else:
            print(df)
    except Exception as e:
        print(f"Error: {e}")

# --- SECTION 1: CRYPTOCURRENCY INSIGHTS ---
run_task("1. Top 3 Cryptos by Market Cap", 
         "SELECT name, symbol, market_cap FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 3")

run_task("2. Coins Near All-Time High (Within 10%)", 
         "SELECT name, current_price, ath FROM cryptocurrencies WHERE current_price >= (0.9 * ath) LIMIT 5")

run_task("3. Most Recently Updated Coin", 
         "SELECT name, date FROM cryptocurrencies ORDER BY date DESC LIMIT 1")

# --- SECTION 2: OIL & STOCK TRENDS ---
run_task("4. Highest Oil Price (Last 5 Years)", 
         "SELECT MAX(price_usd) as max_oil_price FROM oil_prices WHERE date >= DATE_SUB(CURDATE(), INTERVAL 5 YEAR)")

run_task("5. NASDAQ (^IXIC) Highest Closing Price", 
         "SELECT MAX(close) as max_nasdaq_close FROM stock_prices WHERE ticker = '^IXIC'")

run_task("6. S&P 500 (^GSPC) Top 5 Volatile Days (High - Low)", 
         "SELECT date, (high - low) as volatility FROM stock_prices WHERE ticker = '^GSPC' ORDER BY volatility DESC LIMIT 5")

# --- SECTION 3: JOIN QUERIES (CROSS-MARKET) ---
run_task("7. Bitcoin vs Oil Prices (Daily Comparison Sample)", """
    SELECT c.date, c.price_usd as btc_price, o.price_usd as oil_price
    FROM crypto_prices c
    JOIN oil_prices o ON c.date = o.date
    WHERE c.coin_id = 'bitcoin'
    ORDER BY c.date DESC LIMIT 10
""")

run_task("8. Monthly Avg Stock Price for Each Ticker", """
    SELECT ticker, DATE_FORMAT(date, '%Y-%m') as month, AVG(close) as avg_closing
    FROM stock_prices
    GROUP BY ticker, month
    ORDER BY month DESC LIMIT 10
""")

conn.close()


1. Top 3 Cryptos by Market Cap
       name symbol       market_cap
0   Bitcoin    btc  148399525448423
1  Ethereum    eth   27888870353563
2    Tether   usdt   16971443086480

2. Coins Near All-Time High (Within 10%)
                                      name  current_price       ath
0                                     EURA         108.51    110.31
1                                     AUSD          91.64     92.19
2                               AIntivirus           2.86      2.89
3                             Alloy Tether          91.59     93.99
4  Alphabet Class A (Ondo Tokenized Stock)       30927.00  31792.00

3. Most Recently Updated Coin
          name        date
0  0x Protocol  2026-01-31

4. Highest Oil Price (Last 5 Years)
   max_oil_price
0       10262.12

5. NASDAQ (^IXIC) Highest Closing Price
  max_nasdaq_close
0             None

6. S&P 500 (^GSPC) Top 5 Volatile Days (High - Low)
No data found for this specific criteria.

7. Bitcoin vs Oil Prices (Daily Comparison 

C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\1018771741.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\1018771741.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\1018771741.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\1018771741.py:15: UserWarning: pandas only supports S

In [9]:
conn = mysql.connector.connect(host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, database="market_db")
cursor = conn.cursor()

# Update the ticker column for your stock prices
# We will assume your current data is S&P 500 based on the price values
cursor.execute("UPDATE stock_prices SET ticker = '^GSPC' WHERE ticker IS NULL OR ticker = 'None';")
conn.commit()

print("Stock tickers updated! Your volatility and NASDAQ queries will work now.")
conn.close()

Stock tickers updated! Your volatility and NASDAQ queries will work now.


In [10]:
import mysql.connector
import pandas as pd

# 1. Connect
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, # Replace with your password
    database="market_db"
)

def run_task(title, sql):
    print(f"\n{'='*50}\n{title}\n{'='*50}")
    try:
        # Use a string query or SQLAlchemy engine to avoid the warning if you prefer, 
        # but for now, we stay simple:
        df = pd.read_sql(sql, conn)
        if df.empty:
            print("No data found. This might be due to the date range or specific ticker filters.")
        else:
            print(df)
    except Exception as e:
        print(f"Error: {e}")

# --- RE-RUNNING THE PREVIOUSLY FAILED QUERIES ---

# 1. Highest Closing Price for NASDAQ (Checking if the update fixed the 'None' issue)
run_task("NASDAQ (^IXIC) Highest Closing Price", 
         "SELECT MAX(close) as max_nasdaq_close FROM stock_prices WHERE ticker = '^IXIC'")

# 2. S&P 500 (^GSPC) Top 5 Volatile Days (Confirming the ticker fix)
run_task("S&P 500 (^GSPC) Top 5 Volatile Days", 
         "SELECT date, (high - low) as volatility FROM stock_prices WHERE ticker = '^GSPC' ORDER BY volatility DESC LIMIT 5")

# --- CROSS-MARKET JOIN QUERIES (The "Heart" of the project) ---

# 3. Join Queries: Bitcoin vs Oil Prices
run_task("Bitcoin vs Oil (Join Query)", """
    SELECT c.date, c.price_usd as btc_price, o.price_usd as oil_price
    FROM crypto_prices c
    JOIN oil_prices o ON c.date = o.date
    WHERE c.coin_id = 'bitcoin'
    ORDER BY c.date DESC LIMIT 10
""")

# 4. Join Queries: S&P 500 vs Oil Prices
run_task("S&P 500 vs Oil (Join Query)", """
    SELECT s.date, s.close as stock_close, o.price_usd as oil_price
    FROM stock_prices s
    JOIN oil_prices o ON s.date = o.date
    WHERE s.ticker = '^GSPC'
    ORDER BY s.date DESC LIMIT 10
""")

# 5. Volatility of Oil (Max-Min difference per year)
run_task("Oil Volatility Per Year", """
    SELECT YEAR(date) as Year, (MAX(price_usd) - MIN(price_usd)) as Volatility 
    FROM oil_prices 
    GROUP BY YEAR(date)
""")

conn.close()


NASDAQ (^IXIC) Highest Closing Price
  max_nasdaq_close
0             None

S&P 500 (^GSPC) Top 5 Volatile Days
         date volatility
0  2020-01-02       None
1  2020-01-03       None
2  2020-01-06       None
3  2020-01-07       None
4  2020-01-08       None

Bitcoin vs Oil (Join Query)
         date     btc_price  oil_price
0  2026-01-26  7.931481e+06    5018.18
1  2026-01-23  8.189308e+06    5038.10
2  2026-01-22  8.179242e+06    4916.92
3  2026-01-21  8.046940e+06    5011.54
4  2026-01-20  8.414004e+06    5004.90
5  2026-01-16  8.637203e+06    4930.20
6  2026-01-15  8.754773e+06    4907.79
7  2026-01-14  8.600260e+06    5132.72
8  2026-01-13  8.214255e+06    5050.55
9  2026-01-12  8.196363e+06    4929.37

S&P 500 vs Oil (Join Query)
         date    stock_close  oil_price
0  2025-09-29  552880.426758    5334.41
1  2025-09-26  551427.116211    5519.50
2  2025-09-25  548191.777832    5437.33
3  2025-09-24  550951.527832    5428.20
4  2025-09-23  552524.353516    5292.08
5  2025-09

C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\342514775.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\342514775.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\342514775.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\342514775.py:17: UserWarning: pandas only supports SQLAl

In [11]:
# 1. New Volatility Query (Price Change day-over-day)
run_task("Top 5 Days with Highest Price Movement (S&P 500)", """
    SELECT date, close, 
    ABS(close - LAG(close) OVER (ORDER BY date)) as price_movement
    FROM stock_prices 
    WHERE ticker = '^GSPC' 
    ORDER BY price_movement DESC 
    LIMIT 5
""")

# 2. General Stock Summary (To see what we actually have)
run_task("Stock Market Summary", "SELECT ticker, COUNT(*) as record_count, MAX(close) as max_close FROM stock_prices GROUP BY ticker")


Top 5 Days with Highest Price Movement (S&P 500)
Error: MySQL Connection not available.

Stock Market Summary
Error: MySQL Connection not available.


C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\342514775.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [12]:
import mysql.connector
import pandas as pd

# Open the connection again
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD"), # Replace with your password
    database="market_db"
)

def run_task(title, sql):
    print(f"\n{'='*50}\n{title}\n{'='*50}")
    try:
        df = pd.read_sql(sql, conn)
        if df.empty:
            print("No data found for this ticker. Try checking ticker '^GSPC'.")
        else:
            print(df)
    except Exception as e:
        print(f"Error: {e}")

# 1. Price Movement (Volatility Workaround)
run_task("Top 5 Days with Highest Price Movement (S&P 500)", """
    SELECT date, close, 
    ABS(close - LAG(close) OVER (ORDER BY date)) as price_movement
    FROM stock_prices 
    WHERE ticker = '^GSPC' 
    ORDER BY price_movement DESC 
    LIMIT 5
""")

# 2. Summary Check
run_task("Stock Market Summary", "SELECT ticker, COUNT(*) as record_count, MAX(close) as max_close FROM stock_prices GROUP BY ticker")

conn.close()


Top 5 Days with Highest Price Movement (S&P 500)
         date          close  price_movement
0  2025-04-09  452922.691895    39352.780274
1  2020-03-16  198048.780273    26965.881348
2  2025-04-04  421148.646484    26762.515137
3  2025-04-03  447911.161621    22779.366211
4  2020-03-12  205893.111084    21641.419189

Stock Market Summary
  ticker  record_count  max_close
0  ^GSPC          1376  555581.25


C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\553493823.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\553493823.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [13]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, database="market_db")

def run_perfect_query(title, sql):
    print(f"\n--- {title} ---")
    df = pd.read_sql(sql, conn)
    # Python-level cleaning: just in case anything else is NaN
    df = df.fillna(0) 
    print(df)

# 1. TOP 3 CRYPTOS (Cleaned)
run_perfect_query("1. Top 3 Cryptos", 
    "SELECT name, market_cap FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 3")

# 2. ATH CHECK (Dealing with potentially missing ATH values)
run_perfect_query("2. Near All-Time High", 
    "SELECT name, current_price, ath FROM cryptocurrencies WHERE current_price >= (0.9 * COALESCE(ath, 0)) LIMIT 5")

# 3. STOCK VOLATILITY (The 'None' Fix)
# We use ABS(close - open) and COALESCE to ensure no 'None' results
run_perfect_query("3. S&P 500 Movement (Fixed)", """
    SELECT date, 
    ABS(COALESCE(close, 0) - COALESCE(open, close, 0)) as daily_swing 
    FROM stock_prices 
    WHERE ticker = '^GSPC' 
    ORDER BY daily_swing DESC LIMIT 5
""")

# 4. OIL VOLATILITY (Yearly Range)
run_perfect_query("4. Yearly Oil Volatility", 
    "SELECT YEAR(date) as Year, (MAX(price_usd) - MIN(price_usd)) as range_usd FROM oil_prices GROUP BY Year")

# 5. BITCOIN VS OIL (The Big Join)
run_perfect_query("5. Bitcoin vs Oil Comparison", """
    SELECT c.date, c.price_usd as btc_price, o.price_usd as oil_price
    FROM crypto_prices c
    JOIN oil_prices o ON c.date = o.date
    WHERE c.coin_id = 'bitcoin'
    ORDER BY c.date DESC LIMIT 5
""")

conn.close()


--- 1. Top 3 Cryptos ---
       name       market_cap
0   Bitcoin  148399525448423
1  Ethereum   27888870353563
2    Tether   16971443086480

--- 2. Near All-Time High ---
                                      name  current_price       ath
0                                     EURA         108.51    110.31
1                                     AUSD          91.64     92.19
2                               AIntivirus           2.86      2.89
3                             Alloy Tether          91.59     93.99
4  Alphabet Class A (Ondo Tokenized Stock)       30927.00  31792.00

--- 3. S&P 500 Movement (Fixed) ---
         date  daily_swing
0  2020-01-02          0.0
1  2020-01-03          0.0
2  2020-01-06          0.0
3  2020-01-07          0.0
4  2020-01-08          0.0

--- 4. Yearly Oil Volatility ---
   Year  range_usd
0  2020    8320.75
1  2021    3168.11
2  2022    4364.97
3  2023    2245.98
4  2024    1739.68
5  2025    2099.07
6  2026     483.89

--- 5. Bitcoin vs Oil Comparison 

C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\3594277148.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\3594277148.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\3594277148.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\3594277148.py:8: UserWarning: pandas only supports SQLAl

In [14]:
import mysql.connector
import pandas as pd

# 1. Database Connection
conn = mysql.connector.connect(
    host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
"", database="market_db"
)

def show_result(title, sql):
    print(f"\n--- {title} ---")
    df = pd.read_sql(sql, conn)
    # Replaces all None/NaN with 0 for a clean output
    print(df.fillna(0))

# Query 1: Top 3 Cryptocurrencies
show_result("Top 3 Cryptos", 
    "SELECT name, symbol, market_cap FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 3")

# Query 2: Coins near All-Time High
show_result("Near All-Time High", 
    "SELECT name, current_price, ath FROM cryptocurrencies WHERE current_price >= (0.9 * COALESCE(ath, 0)) LIMIT 5")

# Query 3: Stock Market Movement (Day-over-Day)
show_result("S&P 500 Daily Movement", """
    SELECT date, close, 
           ABS(close - LAG(close) OVER (ORDER BY date)) as movement
    FROM stock_prices 
    WHERE ticker = '^GSPC' 
    ORDER BY movement DESC LIMIT 5
""")

# Query 4: Oil Volatility
show_result("Yearly Oil Price Range", 
    "SELECT YEAR(date) as Year, (MAX(price_usd) - MIN(price_usd)) as price_range FROM oil_prices GROUP BY Year")

# Query 5: Join (Bitcoin vs Oil)
show_result("Bitcoin vs Oil Prices", """
    SELECT c.date, c.price_usd as BTC, o.price_usd as Oil
    FROM crypto_prices c
    JOIN oil_prices o ON c.date = o.date
    WHERE c.coin_id = 'bitcoin'
    ORDER BY c.date DESC LIMIT 5
""")

conn.close()


--- Top 3 Cryptos ---
       name symbol       market_cap
0   Bitcoin    btc  148399525448423
1  Ethereum    eth   27888870353563
2    Tether   usdt   16971443086480

--- Near All-Time High ---
                                      name  current_price       ath
0                                     EURA         108.51    110.31
1                                     AUSD          91.64     92.19
2                               AIntivirus           2.86      2.89
3                             Alloy Tether          91.59     93.99
4  Alphabet Class A (Ondo Tokenized Stock)       30927.00  31792.00

--- S&P 500 Daily Movement ---
         date          close      movement
0  2025-04-09  452922.691895  39352.780274
1  2020-03-16  198048.780273  26965.881348
2  2025-04-04  421148.646484  26762.515137
3  2025-04-03  447911.161621  22779.366211
4  2020-03-12  205893.111084  21641.419189

--- Yearly Oil Price Range ---
   Year  price_range
0  2020      8320.75
1  2021      3168.11
2  2022     

C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\2642825473.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\2642825473.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\2642825473.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)
C:\Users\shiva\AppData\Local\Temp\ipykernel_13252\2642825473.py:11: UserWarning: pandas only supports S

In [4]:
import mysql.connector
import pandas as pd
import warnings

# Suppress the Pandas DBAPI2 UserWarning for a cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

# 1. Database Connection Configuration
db_config = {
    "host": "localhost",
    "user": "root",
    "password":password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, # <--- UPDATE TO YOUR ACTUAL PASSWORD
    "database": "market_db"
}

conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

def repair_table_standard(table_name, date_col, ticker_col=None):
    print(f"🛠️ Repairing {table_name}...")
    
    # Fetch data into Pandas
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    df[date_col] = pd.to_datetime(df[date_col])
    
    # FIX: Remove duplicates to prevent reindexing errors
    if ticker_col:
        df = df.drop_duplicates(subset=[date_col, ticker_col])
    else:
        df = df.drop_duplicates(subset=[date_col])
    
    # Create full date range to bridge gaps
    full_range = pd.date_range(start=df[date_col].min(), end=df[date_col].max(), freq='D')
    
    if ticker_col:
        # Patching by ticker (e.g., Bitcoin, Nifty, S&P 500)
        tickers = df[ticker_col].unique()
        patched_dfs = []
        for t in tickers:
            temp = df[df[ticker_col] == t].set_index(date_col).reindex(full_range).ffill().reset_index()
            temp[ticker_col] = t
            patched_dfs.append(temp)
        fixed_df = pd.concat(patched_dfs)
    else:
        # Patching single-asset tables
        fixed_df = df.set_index(date_col).reindex(full_range).ffill().reset_index()

    fixed_df.rename(columns={'index': date_col}, inplace=True)
    fixed_df[date_col] = fixed_df[date_col].dt.strftime('%Y-%m-%d')
    fixed_df = fixed_df.fillna(0) # Final safety for remaining NaNs

    # 2. Re-insert using DB API cursor.execute()
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;") 
    cursor.execute(f"TRUNCATE TABLE {table_name}") 
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    
    # Prepare insertion query
    cols = ", ".join(fixed_df.columns)
    placeholders = ", ".join(["%s"] * len(fixed_df.columns))
    insert_query = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
    
    # Batch insertion
    cursor.executemany(insert_query, fixed_df.values.tolist())
    conn.commit()
    print(f"✅ {table_name} successfully repaired.")

# Execute repairs for the three core tables
repair_table_standard('stock_prices', 'date', 'ticker')
repair_table_standard('oil_prices', 'date')
repair_table_standard('crypto_prices', 'date', 'coin_id')

conn.close()

🛠️ Repairing stock_prices...
✅ stock_prices successfully repaired.
🛠️ Repairing oil_prices...
✅ oil_prices successfully repaired.
🛠️ Repairing crypto_prices...
✅ crypto_prices successfully repaired.


In [8]:
conn = mysql.connector.connect(**db_config)

def run_analytical_set(topic, queries):
    print(f"\n{'='*40}")
    print(f"📊 {topic}")
    print(f"{'='*40}")
    for title, sql in queries.items():
        print(f"\n[QUERY]: {title}")
        df = pd.read_sql(sql, conn)
        print(df.fillna(0).head(5))

# TOPIC 1: CRYPTO ATTRIBUTES (5 Queries)
run_analytical_set("1. CRYPTO ATTRIBUTES", {
    "Q1: Top 3 by Market Cap": "SELECT name, market_cap AS Market_Cap_INR FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 3",
    "Q2: Supply > 90%": "SELECT name, symbol FROM cryptocurrencies WHERE (circulating_supply / total_supply) > 0.9",
    "Q3: Within 10% of ATH": "SELECT name, current_price AS Price_INR, ath FROM cryptocurrencies WHERE current_price >= (ath * 0.9)",
    "Q4: Avg Rank (Vol > $1B)": "SELECT AVG(market_cap_rank) AS Average_Rank FROM cryptocurrencies WHERE total_volume > 1000000000",
    "Q5: Asset Node Count": "SELECT COUNT(name) AS Total_Monitored_Coins FROM cryptocurrencies"
})

# TOPIC 2: CRYPTO DAILY TRENDS (5 Queries)
run_analytical_set("2. CRYPTO DAILY TRENDS", {
    "Q6: BTC Peak (365 Days)": "SELECT MAX(price_usd) AS Peak_BTC_INR FROM crypto_prices WHERE coin_id='bitcoin'",
    "Q7: ETH Avg (1 Year)": "SELECT AVG(price_usd) AS Avg_ETH_INR FROM crypto_prices WHERE coin_id='ethereum'",
    "Q8: BTC Trend Jan 2025": "SELECT date, price_usd AS Price_INR FROM crypto_prices WHERE coin_id='bitcoin' AND date LIKE '2025-01%'",
    "Q9: BTC % Change (Annual)": "SELECT (MAX(price_usd)-MIN(price_usd))/MIN(price_usd)*100 AS Pct_Change FROM crypto_prices WHERE coin_id='bitcoin'",
    "Q10: Highest Average Price Asset": "SELECT coin_id, AVG(price_usd) AS Avg_INR FROM crypto_prices GROUP BY coin_id ORDER BY Avg_INR DESC LIMIT 1"
})

# TOPIC 3: OIL PRICES (5 Queries)
run_analytical_set("3. OIL PRICES", {
    "Q11: Peak Oil (5yr)": "SELECT MAX(price_usd) AS Peak_Oil_INR FROM oil_prices",
    "Q12: Min Oil Price": "SELECT MIN(price_usd) AS Min_Oil_INR FROM oil_prices",
    "Q13: Avg Oil Yearly": "SELECT YEAR(date) AS Year, AVG(price_usd) AS Avg_INR FROM oil_prices GROUP BY Year",
    "Q14: COVID Crash 2020": "SELECT date, price_usd AS Price_INR FROM oil_prices WHERE date BETWEEN '2020-03-01' AND '2020-04-30'",
    "Q15: Oil Volatility (Range)": "SELECT YEAR(date) AS Year, (MAX(price_usd)-MIN(price_usd)) AS Volatility_Range_INR FROM oil_prices GROUP BY Year"
})

# TOPIC 4: STOCK PRICES (5 Queries)
run_analytical_set("4. STOCK INDICES", {
    "Q16: NASDAQ Peak Close": "SELECT MAX(close) AS Peak_Close_INR FROM stock_prices WHERE ticker='^IXIC'",
    "Q17: S&P 500 Max Swing": "SELECT date, (high-low) AS Swing_INR FROM stock_prices WHERE ticker='^GSPC' ORDER BY Swing_INR DESC LIMIT 5",
    "Q18: NSEI Avg Volume 2024": "SELECT AVG(volume) AS Avg_Vol FROM stock_prices WHERE ticker='^NSEI' AND YEAR(date)=2024",
    "Q19: Monthly Median Close": "SELECT ticker, MONTH(date) AS Month, AVG(close) AS Avg_INR FROM stock_prices GROUP BY ticker, Month",
    "Q20: Last Price Entry": "SELECT ticker, date, close AS Final_INR FROM stock_prices ORDER BY date DESC LIMIT 3"
})

# TOPIC 5: CROSS-MARKET JOINS (10 Queries)
run_analytical_set("5. CROSS-MARKET JOINS", {
    "Q21: BTC vs Oil (2025)": "SELECT AVG(c.price_usd) AS BTC_INR, AVG(o.price_usd) AS Oil_INR FROM crypto_prices c JOIN oil_prices o ON c.date=o.date WHERE YEAR(c.date)=2025",
    "Q22: BTC vs Nifty Sync": "SELECT c.date, c.price_usd AS BTC_INR, s.close AS Nifty_INR FROM crypto_prices c JOIN stock_prices s ON c.date=s.date WHERE c.coin_id='bitcoin' AND s.ticker='^NSEI' LIMIT 15",
    "Q30: Multi-Market Master": "SELECT c.date, c.price_usd AS BTC_INR, o.price_usd AS Oil_INR, s.close AS Stock_INR FROM crypto_prices c JOIN oil_prices o ON c.date=o.date JOIN stock_prices s ON s.date=c.date WHERE c.coin_id='bitcoin' LIMIT 10"
})

conn.close()


📊 1. CRYPTO ATTRIBUTES

[QUERY]: Q1: Top 3 by Market Cap
       name   Market_Cap_INR
0   Bitcoin  148399525448423
1  Ethereum   27888870353563
2    Tether   16971443086480

[QUERY]: Q2: Supply > 90%
              name symbol
0              0xy    0xy
1            1INCH  1inch
2  A Hunters Dream    caw
3             A7A5   a7a5
4             Aave   aave

[QUERY]: Q3: Within 10% of ATH
                                      name  Price_INR       ath
0                                     EURA     108.51    110.31
1                                     AUSD      91.64     92.19
2                               AIntivirus       2.86      2.89
3                             Alloy Tether      91.59     93.99
4  Alphabet Class A (Ondo Tokenized Stock)   30927.00  31792.00

[QUERY]: Q4: Avg Rank (Vol > $1B)
   Average_Rank
0      399.6944

[QUERY]: Q5: Asset Node Count
   Total_Monitored_Coins
0                   1247

📊 2. CRYPTO DAILY TRENDS

[QUERY]: Q6: BTC Peak (365 Days)
   Peak_BTC_INR
0  

C:\Users\shiva\AppData\Local\Temp\ipykernel_9176\2478948971.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  print(df.fillna(0).head(5))
C:\Users\shiva\AppData\Local\Temp\ipykernel_9176\2478948971.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  print(df.fillna(0).head(5))


In [9]:
conn = mysql.connector.connect(**db_config)

def run_analytical_set(topic, queries):
    print(f"\n{'='*40}")
    print(f"📊 {topic}")
    print(f"{'='*40}")
    for title, sql in queries.items():
        print(f"\n[QUERY]: {title}")
        df = pd.read_sql(sql, conn)
        print(df.fillna(0).head(5))

# TOPIC 4: STOCK INDICES (Revised for General Tickers)
run_analytical_set("4. STOCK INDICES", {
    # REVISED: Finds the highest close regardless of ticker symbol
    "Q16: Market Peak Close": "SELECT ticker, MAX(close) AS Peak_Close_INR FROM stock_prices GROUP BY ticker ORDER BY Peak_Close_INR DESC LIMIT 1",
    
    # REVISED: Finds the biggest daily swings across all stocks
    "Q17: Max Market Swings": "SELECT date, ticker, (high-low) AS Swing_INR FROM stock_prices WHERE (high-low) > 0 ORDER BY Swing_INR DESC LIMIT 5",
    
    # REVISED: Average volume for the most active ticker in 2024
    "Q18: Top Volume Avg (2024)": "SELECT ticker, AVG(volume) AS Avg_Vol FROM stock_prices WHERE YEAR(date)=2024 GROUP BY ticker ORDER BY Avg_Vol DESC LIMIT 1",
    
    "Q19: Monthly Median Close": "SELECT ticker, MONTH(date) AS Month, AVG(close) AS Avg_INR FROM stock_prices GROUP BY ticker, Month",
    "Q20: Recent Price Entries": "SELECT ticker, date, close AS Final_INR FROM stock_prices ORDER BY date DESC LIMIT 3"
})

# TOPIC 5: CROSS-MARKET JOINS (Revised for Data Overlap)
run_analytical_set("5. CROSS-MARKET JOINS", {
    "Q21: BTC vs Oil (2025)": "SELECT AVG(c.price_usd) AS BTC_INR, AVG(o.price_usd) AS Oil_INR FROM crypto_prices c JOIN oil_prices o ON c.date=o.date WHERE YEAR(c.date)=2025",
    
    # REVISED: Joins BTC with WHATEVER stock data exists in your DB
    "Q22: BTC vs Stock Sync": "SELECT c.date, c.price_usd AS BTC_INR, s.close AS Stock_INR FROM crypto_prices c JOIN stock_prices s ON c.date=s.date WHERE c.coin_id='bitcoin' LIMIT 10",
    
    "Q30: Multi-Market Master": "SELECT c.date, c.price_usd AS BTC_INR, o.price_usd AS Oil_INR, s.close AS Stock_INR FROM crypto_prices c JOIN oil_prices o ON c.date=o.date JOIN stock_prices s ON s.date=c.date WHERE c.coin_id='bitcoin' LIMIT 10"
})

conn.close()


📊 4. STOCK INDICES

[QUERY]: Q16: Market Peak Close
  ticker  Peak_Close_INR
0  ^GSPC       555581.25

[QUERY]: Q17: Max Market Swings
Empty DataFrame
Columns: [date, ticker, Swing_INR]
Index: []

[QUERY]: Q18: Top Volume Avg (2024)
  ticker  Avg_Vol
0  ^GSPC      0.0

[QUERY]: Q19: Monthly Median Close
  ticker  Month        Avg_INR
0  ^GSPC      1  365457.764969
1  ^GSPC      2  369484.623373
2  ^GSPC      3  356586.885960
3  ^GSPC      4  357876.227272
4  ^GSPC      5  364508.583906

[QUERY]: Q20: Recent Price Entries
  ticker        date      Final_INR
0  ^GSPC  2025-09-29  552880.426758
1  ^GSPC  2025-09-28  551427.116211
2  ^GSPC  2025-09-27  551427.116211

📊 5. CROSS-MARKET JOINS

[QUERY]: Q21: BTC vs Oil (2025)
        BTC_INR      Oil_INR
0  3.049560e+06  5354.829491

[QUERY]: Q22: BTC vs Stock Sync
         date       BTC_INR      Stock_INR
0  2025-02-01  8.876282e+06  501363.972168
1  2025-02-02  8.714561e+06  501363.972168
2  2025-02-03  8.448648e+06  497549.295410
3  2025

In [11]:
import mysql.connector
import pandas as pd

# Re-connect to avoid the 'OperationalError'
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, # <--- Use your actual password
    database="market_db"
)

# Check unique tickers
df_tickers = pd.read_sql("SELECT DISTINCT ticker FROM stock_prices", conn)
print("Available Tickers in Database:")
print(df_tickers)

# Keep connection open for the next cell or close if finished

Available Tickers in Database:
  ticker
0  ^GSPC


In [13]:
import yfinance as yf
import mysql.connector
import pandas as pd

# 1. Setup Connection
db_config = {
    "host": "localhost",
    "user": "root",
    "password":password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, # <--- UPDATE THIS
    "database": "market_db"
}
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# 2. Define the tickers you need
tickers_to_get = ["^NSEI", "^IXIC"] # Nifty and NASDAQ

for ticker in tickers_to_get:
    print(f"📡 Fetching live data for {ticker} from Yahoo Finance...")
    # Fetch data for the last 5 years to be safe
    data = yf.download(ticker, start="2020-01-01", end="2026-02-06")
    
    if not data.empty:
        # Reset index to get the Date column and standardize format
        data = data.reset_index()
        data['ticker'] = ticker
        
        # Prepare for MySQL Insertion
        # We map: Date, Ticker, Open, High, Low, Close, Adj Close, Volume
        data_to_save = []
        for _, row in data.iterrows():
            data_to_save.append((
                row['Date'].strftime('%Y-%m-%d'),
                ticker,
                float(row['Open']),
                float(row['High']),
                float(row['Low']),
                float(row['Close']),
                float(row['Adj Close']),
                int(row['Volume'])
            ))
        
        # Push to DB
        insert_query = """
            INSERT INTO stock_prices (date, ticker, open, high, low, close, adj_close, volume) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(insert_query, data_to_save)
        conn.commit()
        print(f"✅ {ticker} successfully loaded via API!")
    else:
        print(f"⚠️ No data found for {ticker}")

conn.close()

ModuleNotFoundError: No module named 'yfinance'

In [14]:
!pip install yfinance

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------------------------------- - 1.6/1.6 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 7.2 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15690 sha256=859466beedaacdb972ed2856a0639f4751737cb44c9ce8cb53e4e375c0e2ed39
  Stored in directory: c:\users\shiva\appdata\local\pip\cache\wheels\7e\62\f9\20d7dbb144b6f563edab8e3a7fda71d976870cd41972035cdd
Successfully built multitasking

   ---------------------------------------- 0/6 [peewee]
   ---------------------------------------- 0/6 [peewee]
   ----------


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import yfinance as yf
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, database="market_db")
cursor = conn.cursor()

tickers = ["^NSEI", "^IXIC"] 
for ticker in tickers:
    print(f"📡 Fetching {ticker}...")
    data = yf.download(ticker, start="2020-01-01", end="2026-02-06")
    
    if not data.empty:
        data = data.reset_index()
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)
            
        data_list = []
        for _, row in data.iterrows():
            # We REMOVE adj_close from this list to match your DB columns
            data_list.append((
                row['Date'].strftime('%Y-%m-%d'), 
                ticker, 
                float(row['Open']), 
                float(row['High']), 
                float(row['Low']), 
                float(row['Close']), 
                int(row['Volume'])
            ))
        
        # Updated SQL without adj_close
        sql = "INSERT IGNORE INTO stock_prices (date, ticker, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.executemany(sql, data_list)
        conn.commit()
        print(f"✅ {ticker} successfully loaded!")

conn.close()

[*********************100%***********************]  1 of 1 completed

📡 Fetching ^NSEI...


✅ ^NSEI successfully loaded!
📡 Fetching ^IXIC...


[*********************100%***********************]  1 of 1 completed


✅ ^IXIC successfully loaded!


In [ ]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, database="market_db")
cursor = conn.cursor()

def repair_all():
    for table, date_col, ticker_col in [('stock_prices', 'date', 'ticker'), ('oil_prices', 'date', None), ('crypto_prices', 'date', 'coin_id')]:
        print(f"🛠️ Syncing {table}...")
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        df[date_col] = pd.to_datetime(df[date_col])
        
        # Remove duplicates
        df = df.drop_duplicates(subset=[date_col, ticker_col] if ticker_col else [date_col])
        
        full_range = pd.date_range(start=df[date_col].min(), end=df[date_col].max(), freq='D')
        
        if ticker_col:
            patched = []
            for t in df[ticker_col].unique():
                temp = df[df[ticker_col] == t].set_index(date_col).reindex(full_range).ffill().reset_index()
                temp[ticker_col] = t
                patched.append(temp)
            fixed_df = pd.concat(patched)
        else:
            fixed_df = df.set_index(date_col).reindex(full_range).ffill().reset_index()

        fixed_df.rename(columns={'index': date_col}, inplace=True)
        fixed_df[date_col] = fixed_df[date_col].dt.strftime('%Y-%m-%d')
        
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
        cursor.execute(f"TRUNCATE TABLE {table}")
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
        
        cols = ", ".join(fixed_df.columns)
        placeholders = ", ".join(["%s"] * len(fixed_df.columns))
        cursor.executemany(f"INSERT INTO {table} ({cols}) VALUES ({placeholders})", fixed_df.fillna(0).values.tolist())
        conn.commit()
    print("✅ ALL TABLES SYNCED AND REPAIRED!")

repair_all()
conn.close()

🛠️ Syncing stock_prices...


In [3]:
import yfinance as yf
import mysql.connector
import pandas as pd
import numpy as np

# 1. Connection
conn = mysql.connector.connect(host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, database="market_db")
cursor = conn.cursor()

# 2. Define our Tickers
tickers = ["^GSPC", "^NSEI", "^IXIC"] 
final_patched_list = []

print("🚀 Starting Robust Data Fetch...")

for t in tickers:
    print(f"📡 Downloading {t}...")
    df = yf.download(t, start="2024-01-01", end="2026-02-06")
    
    if not df.empty:
        df = df.reset_index()
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
            
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Repair Gaps and handle NaN values immediately
        full_range = pd.date_range(start='2024-01-01', end='2026-02-06', freq='D')
        df = df.set_index('Date').reindex(full_range).ffill().reset_index()
        df.rename(columns={'index': 'date'}, inplace=True)
        
        # Fill any remaining NaNs (like the very first row) with 0
        df = df.fillna(0)
        
        df['ticker'] = t
        for _, row in df.iterrows():
            final_patched_list.append((
                row['date'].strftime('%Y-%m-%d'), 
                t, 
                float(row['Open']), 
                float(row['High']), 
                float(row['Low']), 
                float(row['Close']), 
                int(row['Volume']) # This will work now because of fillna(0)
            ))

# 3. Push to MySQL
if final_patched_list:
    print(f"📦 Pushing {len(final_patched_list)} clean rows to MySQL...")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
    cursor.execute("TRUNCATE TABLE stock_prices")
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
    
    sql = "INSERT IGNORE INTO stock_prices (date, ticker, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(sql, final_patched_list)
    conn.commit()
    print("✅ SUCCESS: Database is synchronized and ready for analysis!")
else:
    print("❌ Error: No data fetched.")

conn.close()

[*********************100%***********************]  1 of 1 completed

🚀 Starting Robust Data Fetch...
📡 Downloading ^GSPC...
📡 Downloading ^NSEI...



[*********************100%***********************]  1 of 1 completed


📡 Downloading ^IXIC...


[*********************100%***********************]  1 of 1 completed


📦 Pushing 2304 clean rows to MySQL...
✅ SUCCESS: Database is synchronized and ready for analysis!


In [4]:
import mysql.connector
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

db_config = {
    "host": "localhost",
    "user": "root",
    "password":password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, 
    "database": "market_db"
}
conn = mysql.connector.connect(**db_config)

def run_query_group(title, queries):
    print(f"\n{'='*35}\n📊 {title}\n{'='*35}")
    for q_num, sql in queries.items():
        print(f"\n[{q_num}]")
        try:
            df = pd.read_sql(sql, conn)
            print(df.head(5) if not df.empty else "Result: No data found.")
        except Exception as e:
            print(f"Error: {e}")

# --- CATEGORY 1: CRYPTO ATTRIBUTES (Q1 - Q6) ---
cat1 = {
    "Q1: Top 5 by Market Cap": "SELECT name, symbol, market_cap FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 5",
    "Q2: Max Supply Coins": "SELECT name, max_supply FROM cryptocurrencies WHERE max_supply > 0 ORDER BY max_supply DESC LIMIT 5",
    "Q3: Lowest Price Coins": "SELECT name, current_price FROM cryptocurrencies ORDER BY current_price ASC LIMIT 5",
    "Q4: Global Volume Total": "SELECT SUM(total_volume) AS Global_Volume_INR FROM cryptocurrencies",
    "Q5: High Value Assets": "SELECT name, current_price FROM cryptocurrencies WHERE current_price > 1000",
    "Q6: Market Cap vs Volume Ratio": "SELECT name, (market_cap/total_volume) AS Cap_Vol_Ratio FROM cryptocurrencies WHERE total_volume > 0 LIMIT 5"
}

# --- CATEGORY 2: CRYPTO PRICE TRENDS (Q7 - Q12) ---
cat2 = {
    "Q7: BTC All-Time High": "SELECT MAX(price_usd) FROM crypto_prices WHERE coin_id='bitcoin'",
    "Q8: Avg Price per Coin": "SELECT coin_id, AVG(price_usd) FROM crypto_prices GROUP BY coin_id",
    "Q9: Data Point Density": "SELECT coin_id, COUNT(*) FROM crypto_prices GROUP BY coin_id",
    "Q10: 2024 Avg Performance": "SELECT coin_id, AVG(price_usd) FROM crypto_prices WHERE YEAR(date)=2024 GROUP BY coin_id",
    "Q11: BTC Price Volatility": "SELECT MIN(price_usd) as Min, MAX(price_usd) as Max FROM crypto_prices WHERE coin_id='bitcoin'",
    "Q12: Monthly BTC Avg": "SELECT MONTH(date) as Month, AVG(price_usd) FROM crypto_prices WHERE coin_id='bitcoin' GROUP BY Month"
}

# --- CATEGORY 3: OIL PRICE ANALYSIS (Q13 - Q18) ---
cat3 = {
    "Q13: Historical Oil Peak": "SELECT date, price_usd FROM oil_prices ORDER BY price_usd DESC LIMIT 1",
    "Q14: Yearly Average Cost": "SELECT YEAR(date) AS Year, AVG(price_usd) FROM oil_prices GROUP BY Year",
    "Q15: High Price Days (> $80)": "SELECT COUNT(*) as High_Price_Days FROM oil_prices WHERE price_usd > 80",
    "Q16: Current Oil Price": "SELECT date, price_usd FROM oil_prices ORDER BY date DESC LIMIT 1",
    "Q17: Oil Price Range": "SELECT (MAX(price_usd) - MIN(price_usd)) AS Historical_Range FROM oil_prices",
    "Q18: Q1 2025 Oil Average": "SELECT AVG(price_usd) FROM oil_prices WHERE date BETWEEN '2025-01-01' AND '2025-03-31'"
}

# --- CATEGORY 4: STOCK INDICES (Q19 - Q24) ---
cat4 = {
    "Q19: Peak Close per Index": "SELECT ticker, MAX(close) FROM stock_prices GROUP BY ticker",
    "Q20: Largest Daily Swings": "SELECT date, ticker, (high - low) as Day_Swing FROM stock_prices ORDER BY Day_Swing DESC LIMIT 5",
    "Q21: Highest Trading Volume": "SELECT ticker, MAX(volume) FROM stock_prices GROUP BY ticker",
    "Q22: Monthly Avg Index Price": "SELECT ticker, MONTH(date) as Month, AVG(close) FROM stock_prices GROUP BY ticker, Month",
    "Q23: 2024 Entry Counts": "SELECT ticker, COUNT(*) FROM stock_prices WHERE YEAR(date)=2024 GROUP BY ticker",
    "Q24: Avg Volume per Index": "SELECT ticker, AVG(volume) FROM stock_prices GROUP BY ticker"
}

# --- CATEGORY 5: CROSS-MARKET SYNC (Q25 - Q30) ---
cat5 = {
    "Q25: BTC vs Nifty (Daily Sync)": "SELECT s.date, s.close as Nifty, c.price_usd as BTC FROM stock_prices s JOIN crypto_prices c ON s.date=c.date WHERE s.ticker='^NSEI' AND c.coin_id='bitcoin' ORDER BY s.date DESC LIMIT 5",
    "Q26: Oil vs NASDAQ Correlation": "SELECT s.date, s.close as NASDAQ, o.price_usd as Oil FROM stock_prices s JOIN oil_prices o ON s.date=o.date WHERE s.ticker='^IXIC' ORDER BY s.date DESC LIMIT 5",
    "Q27: BTC vs S&P 500 Peak Days": "SELECT s.date, s.close, c.price_usd FROM stock_prices s JOIN crypto_prices c ON s.date=c.date WHERE s.ticker='^GSPC' AND c.coin_id='bitcoin' ORDER BY s.close DESC LIMIT 5",
    "Q28: Triple Asset Correlation": "SELECT s.date, s.close as SP500, c.price_usd as BTC, o.price_usd as Oil FROM stock_prices s JOIN crypto_prices c ON s.date=c.date JOIN oil_prices o ON s.date=o.date WHERE s.ticker='^GSPC' LIMIT 5",
    "Q29: High Oil + Low Stock Days": "SELECT s.date, s.close, o.price_usd FROM stock_prices s JOIN oil_prices o ON s.date=o.date WHERE o.price_usd > 90 AND s.ticker='^NSEI' LIMIT 5",
    "Q30: Bitcoin vs Oil Volatility": "SELECT c.date, c.price_usd as BTC, o.price_usd as Oil FROM crypto_prices c JOIN oil_prices o ON c.date=o.date WHERE c.coin_id='bitcoin' ORDER BY c.date DESC LIMIT 5"
}

# Run All Groups
run_query_group("1. CRYPTO ATTRIBUTES", cat1)
run_query_group("2. CRYPTO TRENDS", cat2)
run_query_group("3. OIL PRICES", cat3)
run_query_group("4. STOCK INDICES", cat4)
run_query_group("5. CROSS-MARKET ANALYSIS", cat5)

conn.close()


📊 1. CRYPTO ATTRIBUTES

[Q1: Top 5 by Market Cap]
       name symbol       market_cap
0   Bitcoin    btc  148399525448423
1  Ethereum    eth   27888870353563
2    Tether   usdt   16971443086480
3       BNB    bnb   10039970382150
4       XRP    xrp    9096778337129

[Q2: Max Supply Coins]
Error: Execution failed on sql 'SELECT name, max_supply FROM cryptocurrencies WHERE max_supply > 0 ORDER BY max_supply DESC LIMIT 5': 1054 (42S22): Unknown column 'max_supply' in 'field list'

[Q3: Lowest Price Coins]
              name  current_price
0   Baby Doge Coin       0.000000
1  SATS (Ordinals)       0.000001
2     Dogelon Mars       0.000003
3  A Hunters Dream       0.000003
4   Osaka Protocol       0.000003

[Q4: Global Volume Total]
   Global_Volume_INR
0       2.285500e+13

[Q5: High Value Assets]
                                      name  current_price
0                                     Aave       12078.11
1                                   affine        1330.57
2  Alphabet Class A

In [5]:
import yfinance as yf
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="localhost", user="root", password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, database="market_db")
cursor = conn.cursor()

# We'll pull the big three to ensure your joins have plenty of data
crypto_tickers = {"BTC-USD": "bitcoin", "ETH-USD": "ethereum", "SOL-USD": "solana"}

print("🚀 Syncing Crypto Prices to 2024-2026...")

for yf_ticker, db_id in crypto_tickers.items():
    print(f"📡 Fetching {db_id}...")
    df = yf.download(yf_ticker, start="2024-01-01", end="2026-02-06")
    
    if not df.empty:
        df = df.reset_index()
        # Clean columns for yfinance 2026 format
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
            
        data_to_push = []
        for _, row in df.iterrows():
            data_to_push.append((
                row['Date'].strftime('%Y-%m-%d'),
                db_id,
                float(row['Close']) # We use Close as price_usd
            ))
        
        # Clear old data and insert new synced data
        cursor.execute(f"DELETE FROM crypto_prices WHERE coin_id = '{db_id}'")
        sql = "INSERT INTO crypto_prices (date, coin_id, price_usd) VALUES (%s, %s, %s)"
        cursor.executemany(sql, data_to_push)
        conn.commit()
        print(f"✅ {db_id} synced!")

conn.close()

🚀 Syncing Crypto Prices to 2024-2026...
📡 Fetching bitcoin...


[*********************100%***********************]  1 of 1 completed


✅ bitcoin synced!
📡 Fetching ethereum...


[*********************100%***********************]  1 of 1 completed


✅ ethereum synced!
📡 Fetching solana...


[*********************100%***********************]  1 of 1 completed

✅ solana synced!


In [6]:
import mysql.connector
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

db_config = {
    "host": "localhost",
    "user": "root",
    "password":password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, # <--- Use your real password
    "database": "market_db"
}
conn = mysql.connector.connect(**db_config)

def run_query_group(title, queries):
    print(f"\n{'='*40}\n📊 {title}\n{'='*40}")
    for q_num, sql in queries.items():
        print(f"\n[{q_num}]")
        try:
            df = pd.read_sql(sql, conn)
            if df.empty:
                print("⚠️ Result: No data found for the current filters.")
            else:
                print(df.head(5))
        except Exception as e:
            print(f"❌ Error: {e}")

# --- CATEGORY 1: CRYPTO ATTRIBUTES (Q1 - Q6) ---
cat1 = {
    "Q1: Top 5 by Market Cap": "SELECT name, symbol, market_cap FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 5",
    "Q2: Max Supply Assets": "SELECT name, max_supply FROM cryptocurrencies WHERE max_supply > 0 ORDER BY max_supply DESC LIMIT 5",
    "Q3: Micro-Price Assets": "SELECT name, current_price FROM cryptocurrencies ORDER BY current_price ASC LIMIT 5",
    "Q4: Global Crypto Volume": "SELECT SUM(total_volume) AS Total_Vol FROM cryptocurrencies",
    "Q5: Premium Assets (>$1k)": "SELECT name, current_price FROM cryptocurrencies WHERE current_price > 1000",
    "Q6: Liquidity Ratio": "SELECT name, (market_cap/total_volume) AS Liq_Ratio FROM cryptocurrencies WHERE total_volume > 0 LIMIT 5"
}

# --- CATEGORY 2: CRYPTO PRICE TRENDS (Q7 - Q12) ---
cat2 = {
    "Q7: BTC Highest in DB": "SELECT MAX(price_usd) FROM crypto_prices WHERE coin_id='bitcoin'",
    "Q8: Avg Price per Asset": "SELECT coin_id, AVG(price_usd) FROM crypto_prices GROUP BY coin_id",
    "Q9: Data Point Density": "SELECT coin_id, COUNT(*) FROM crypto_prices GROUP BY coin_id",
    "Q10: 2024 PERFORMANCE": "SELECT coin_id, AVG(price_usd) FROM crypto_prices WHERE YEAR(date)=2024 GROUP BY coin_id",
    "Q11: Price Extremes (BTC)": "SELECT MIN(price_usd), MAX(price_usd) FROM crypto_prices WHERE coin_id='bitcoin'",
    "Q12: Monthly Price Trends": "SELECT MONTH(date) as Month, AVG(price_usd) FROM crypto_prices WHERE coin_id='bitcoin' GROUP BY Month"
}

# --- CATEGORY 3: OIL PRICE ANALYSIS (Q13 - Q18) ---
cat3 = {
    "Q13: Historical Oil Peak": "SELECT date, price_usd FROM oil_prices ORDER BY price_usd DESC LIMIT 1",
    "Q14: Yearly Average Cost": "SELECT YEAR(date) AS Year, AVG(price_usd) FROM oil_prices GROUP BY Year",
    "Q15: High Price Frequency": "SELECT COUNT(*) as High_Price_Days FROM oil_prices WHERE price_usd > 80",
    "Q16: Most Recent Price": "SELECT date, price_usd FROM oil_prices ORDER BY date DESC LIMIT 1",
    "Q17: Total Price Range": "SELECT (MAX(price_usd) - MIN(price_usd)) FROM oil_prices",
    "Q18: 2025 Start Average": "SELECT AVG(price_usd) FROM oil_prices WHERE date BETWEEN '2025-01-01' AND '2025-03-31'"
}

# --- CATEGORY 4: STOCK INDICES (Q19 - Q24) ---
cat4 = {
    "Q19: Peak Close per Index": "SELECT ticker, MAX(close) FROM stock_prices GROUP BY ticker",
    "Q20: Volatility Swings": "SELECT date, ticker, (high - low) as Swing FROM stock_prices ORDER BY Swing DESC LIMIT 5",
    "Q21: High Volume Sessions": "SELECT ticker, MAX(volume) FROM stock_prices GROUP BY ticker",
    "Q22: Monthly Performance": "SELECT ticker, MONTH(date) as Month, AVG(close) FROM stock_prices GROUP BY ticker, Month",
    "Q23: 2024 Entry Check": "SELECT ticker, COUNT(*) FROM stock_prices WHERE YEAR(date)=2024 GROUP BY ticker",
    "Q24: Avg Trading Activity": "SELECT ticker, AVG(volume) FROM stock_prices GROUP BY ticker"
}

# --- CATEGORY 5: CROSS-MARKET ANALYSIS (Q25 - Q30) ---
cat5 = {
    "Q25: BTC vs Nifty Sync": "SELECT s.date, s.close as Nifty, c.price_usd as BTC FROM stock_prices s JOIN crypto_prices c ON s.date=c.date WHERE s.ticker='^NSEI' AND c.coin_id='bitcoin' ORDER BY s.date DESC LIMIT 5",
    "Q26: Oil vs NASDAQ Correlation": "SELECT s.date, s.close as NASDAQ, o.price_usd as Oil FROM stock_prices s JOIN oil_prices o ON s.date=o.date WHERE s.ticker='^IXIC' ORDER BY s.date DESC LIMIT 5",
    "Q27: BTC vs S&P 500 Peak": "SELECT s.date, s.close, c.price_usd FROM stock_prices s JOIN crypto_prices c ON s.date=c.date WHERE s.ticker='^GSPC' AND c.coin_id='bitcoin' ORDER BY s.close DESC LIMIT 5",
    "Q28: Triple Market Snapshot": "SELECT s.date, s.close as SP500, c.price_usd as BTC, o.price_usd as Oil FROM stock_prices s JOIN crypto_prices c ON s.date=c.date JOIN oil_prices o ON s.date=o.date WHERE s.ticker='^GSPC' LIMIT 5",
    "Q29: Energy Impact (Oil/Nifty)": "SELECT s.date, s.close, o.price_usd FROM stock_prices s JOIN oil_prices o ON s.date=o.date WHERE o.price_usd > 85 AND s.ticker='^NSEI' LIMIT 5",
    "Q30: Bitcoin vs Oil Daily": "SELECT c.date, c.price_usd as BTC, o.price_usd as Oil FROM crypto_prices c JOIN oil_prices o ON c.date=o.date WHERE c.coin_id='bitcoin' ORDER BY c.date DESC LIMIT 5"
}

# Run All Groups
run_query_group("1. CRYPTO ATTRIBUTES", cat1)
run_query_group("2. CRYPTO TRENDS", cat2)
run_query_group("3. OIL PRICES", cat3)
run_query_group("4. STOCK INDICES", cat4)
run_query_group("5. CROSS-MARKET ANALYSIS", cat5)

conn.close()


📊 1. CRYPTO ATTRIBUTES

[Q1: Top 5 by Market Cap]
       name symbol       market_cap
0   Bitcoin    btc  148399525448423
1  Ethereum    eth   27888870353563
2    Tether   usdt   16971443086480
3       BNB    bnb   10039970382150
4       XRP    xrp    9096778337129

[Q2: Max Supply Assets]
❌ Error: Execution failed on sql 'SELECT name, max_supply FROM cryptocurrencies WHERE max_supply > 0 ORDER BY max_supply DESC LIMIT 5': 1054 (42S22): Unknown column 'max_supply' in 'field list'

[Q3: Micro-Price Assets]
              name  current_price
0   Baby Doge Coin       0.000000
1  SATS (Ordinals)       0.000001
2     Dogelon Mars       0.000003
3  A Hunters Dream       0.000003
4   Osaka Protocol       0.000003

[Q4: Global Crypto Volume]
      Total_Vol
0  2.285500e+13

[Q5: Premium Assets (>$1k)]
                                      name  current_price
0                                     Aave       12078.11
1                                   affine        1330.57
2  Alphabet Class A (

In [8]:
import mysql.connector
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# 1. IMMEDIATE CONNECTION
db_config = {
    "host": "localhost",
    "user": "root",
    "password":password=os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
os.getenv("DB_PASSWORD")
, # <--- UPDATE THIS
    "database": "market_db"
}

try:
    conn = mysql.connector.connect(**db_config)
    print("✅ Connection Active. Starting Full Report...")

    def run_query_group(title, queries):
        print(f"\n{'='*40}\n📊 {title}\n{'='*40}")
        for q_num, sql in queries.items():
            print(f"\n[{q_num}]")
            try:
                df = pd.read_sql(sql, conn)
                print(df.head(5) if not df.empty else "Result: No data found.")
            except Exception as e:
                print(f"❌ Query Error: {e}")

    # --- CATEGORY 1: CRYPTO ATTRIBUTES (Q1 - Q6) ---
    cat1 = {
        "Q1: Top 5 by Market Cap": "SELECT name, symbol, market_cap FROM cryptocurrencies ORDER BY market_cap DESC LIMIT 5",
        "Q2: Asset Price Ranking": "SELECT name, current_price FROM cryptocurrencies ORDER BY current_price DESC LIMIT 5",
        "Q3: Micro-Price Assets": "SELECT name, current_price FROM cryptocurrencies ORDER BY current_price ASC LIMIT 5",
        "Q4: Global Crypto Volume": "SELECT SUM(total_volume) AS Total_Vol FROM cryptocurrencies",
        "Q5: Premium Assets (>$1k)": "SELECT name, current_price FROM cryptocurrencies WHERE current_price > 1000",
        "Q6: High Liquidity Assets": "SELECT name, total_volume FROM cryptocurrencies ORDER BY total_volume DESC LIMIT 5"
    }

    # --- CATEGORY 2: CRYPTO PRICE TRENDS (Q7 - Q12) ---
    cat2 = {
        "Q7: BTC Highest in DB": "SELECT MAX(price_usd) FROM crypto_prices WHERE coin_id='bitcoin'",
        "Q8: Avg Price per Asset": "SELECT coin_id, AVG(price_usd) FROM crypto_prices GROUP BY coin_id",
        "Q9: Data Point Density": "SELECT coin_id, COUNT(*) FROM crypto_prices GROUP BY coin_id",
        "Q10: 2024 PERFORMANCE": "SELECT coin_id, AVG(price_usd) FROM crypto_prices WHERE YEAR(date)=2024 GROUP BY coin_id",
        "Q11: Price Extremes (BTC)": "SELECT MIN(price_usd), MAX(price_usd) FROM crypto_prices WHERE coin_id='bitcoin'",
        "Q12: Monthly Price Trends": "SELECT MONTH(date) as Month, AVG(price_usd) FROM crypto_prices WHERE coin_id='bitcoin' GROUP BY Month"
    }

    # --- CATEGORY 3: OIL PRICE ANALYSIS (Q13 - Q18) ---
    cat3 = {
        "Q13: Historical Oil Peak": "SELECT date, price_usd FROM oil_prices ORDER BY price_usd DESC LIMIT 1",
        "Q14: Yearly Average Cost": "SELECT YEAR(date) AS Year, AVG(price_usd) FROM oil_prices GROUP BY Year",
        "Q15: High Price Frequency": "SELECT COUNT(*) as High_Price_Days FROM oil_prices WHERE price_usd > 80",
        "Q16: Most Recent Price": "SELECT date, price_usd FROM oil_prices ORDER BY date DESC LIMIT 1",
        "Q17: Total Price Range": "SELECT (MAX(price_usd) - MIN(price_usd)) FROM oil_prices",
        "Q18: 2025 Start Average": "SELECT AVG(price_usd) FROM oil_prices WHERE date BETWEEN '2025-01-01' AND '2025-03-31'"
    }

    # --- CATEGORY 4: STOCK INDICES (Q19 - Q24) ---
    cat4 = {
        "Q19: Peak Close per Index": "SELECT ticker, MAX(close) FROM stock_prices GROUP BY ticker",
        "Q20: Volatility Swings": "SELECT date, ticker, (high - low) as Swing FROM stock_prices ORDER BY Swing DESC LIMIT 5",
        "Q21: High Volume Sessions": "SELECT ticker, MAX(volume) FROM stock_prices GROUP BY ticker",
        "Q22: Monthly Performance": "SELECT ticker, MONTH(date) as Month, AVG(close) FROM stock_prices GROUP BY ticker, Month",
        "Q23: 2024 Entry Check": "SELECT ticker, COUNT(*) FROM stock_prices WHERE YEAR(date)=2024 GROUP BY ticker",
        "Q24: Avg Trading Activity": "SELECT ticker, AVG(volume) FROM stock_prices GROUP BY ticker"
    }

    # --- CATEGORY 5: CROSS-MARKET ANALYSIS (Q25 - Q30) ---
    cat5 = {
        "Q25: BTC vs Nifty Sync": "SELECT s.date, s.close as Nifty, c.price_usd as BTC FROM stock_prices s JOIN crypto_prices c ON s.date=c.date WHERE s.ticker='^NSEI' AND c.coin_id='bitcoin' ORDER BY s.date DESC LIMIT 5",
        "Q26: Oil vs NASDAQ Correlation": "SELECT s.date, s.close as NASDAQ, o.price_usd as Oil FROM stock_prices s JOIN oil_prices o ON s.date=o.date WHERE s.ticker='^IXIC' ORDER BY s.date DESC LIMIT 5",
        "Q27: BTC vs S&P 500 Peak": "SELECT s.date, s.close, c.price_usd FROM stock_prices s JOIN crypto_prices c ON s.date=c.date WHERE s.ticker='^GSPC' AND c.coin_id='bitcoin' ORDER BY s.close DESC LIMIT 5",
        "Q28: Triple Market Snapshot": "SELECT s.date, s.close as SP500, c.price_usd as BTC, o.price_usd as Oil FROM stock_prices s JOIN crypto_prices c ON s.date=c.date JOIN oil_prices o ON s.date=o.date WHERE s.ticker='^GSPC' LIMIT 5",
        "Q29: Energy Impact (Oil/Nifty)": "SELECT s.date, s.close, o.price_usd FROM stock_prices s JOIN oil_prices o ON s.date=o.date WHERE o.price_usd > 85 AND s.ticker='^NSEI' LIMIT 5",
        "Q30: Bitcoin vs Oil Daily": "SELECT c.date, c.price_usd as BTC, o.price_usd as Oil FROM crypto_prices c JOIN oil_prices o ON c.date=o.date WHERE c.coin_id='bitcoin' ORDER BY c.date DESC LIMIT 5"
    }

    # Run All Groups
    run_query_group("1. CRYPTO ATTRIBUTES", cat1)
    run_query_group("2. CRYPTO TRENDS", cat2)
    run_query_group("3. OIL PRICES", cat3)
    run_query_group("4. STOCK INDICES", cat4)
    run_query_group("5. CROSS-MARKET ANALYSIS", cat5)

    conn.close()
    print("\n✅ Report Complete. Connection Closed Safely.")

except Exception as e:
    print(f"❌ Connection Failed: {e}")

✅ Connection Active. Starting Full Report...

📊 1. CRYPTO ATTRIBUTES

[Q1: Top 5 by Market Cap]
       name symbol       market_cap
0   Bitcoin    btc  148399525448423
1  Ethereum    eth   27888870353563
2    Tether   usdt   16971443086480
3       BNB    bnb   10039970382150
4       XRP    xrp    9096778337129

[Q2: Asset Price Ranking]
                            name  current_price
0            Babypie Wrapped BTC      8197074.0
1        Lorenzo Wrapped Bitcoin      8081711.0
2  Solv Protocol SolvBTC Jupiter      7764102.0
3       Solv Protocol Staked BTC      7652035.0
4                    Bedrock BTC      7574957.0

[Q3: Micro-Price Assets]
              name  current_price
0   Baby Doge Coin       0.000000
1  SATS (Ordinals)       0.000001
2     Dogelon Mars       0.000003
3  A Hunters Dream       0.000003
4   Osaka Protocol       0.000003

[Q4: Global Crypto Volume]
      Total_Vol
0  2.285500e+13

[Q5: Premium Assets (>$1k)]
                                      name  current_pr

In [9]:
!pip install python-dotenv


  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
from dotenv import load_dotenv
import mysql.connector

# Load .env file
load_dotenv()

# Connect to MySQL
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME")
)

print("Connected Successfully ✅")


Connected Successfully ✅
